In [ ]:
import os
import pandas as pd
import glob
import cv2 as cv
import numpy as np
import pandas as pd
from path import Path
from tqdm import tqdm
import nibabel as nib
import tensorflow as tf
from keras import layers
from tensorflow import keras
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import requests
import random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from pydicom.pixel_data_handlers.util import apply_voi_lut
from keras_preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda,Flatten,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras import regularizers
from tensorflow.keras.applications import EfficientNetB3
import torch
import torchvision
from torch.utils.data import Dataset,DataLoader
import math
from tensorflow.keras.callbacks import TensorBoard
from tqdm.notebook import tqdm
import time
from keras.utils.vis_utils import plot_model

In [ ]:
#             0           1        2         3          4          5
names = ['chickenpox','cowpox','healthy','measles','monkeypox','smallpox']

In [ ]:
aug_path= r"D:\Projects\Kaggle\Monkeypox\augmented_images"
pre_path = r"D:\Projects\Kaggle\Monkeypox\preprocessed_original_images"

In [ ]:
class File():
    def __init__(self,path,type) -> None:
        self.path = path
        self.type =type

In [ ]:
data= []

for name in names:
    sec = name+"\*.jpg"
    images_path= os.path.abspath(os.path.join(aug_path,sec))
    files = glob.glob(images_path)
    for index,file in enumerate(files):
        data.append(File(file,name))
    images_path= os.path.abspath(os.path.join(pre_path,sec))
    files = glob.glob(images_path)
    for index,file in enumerate(files):
        data.append(File(file,name))


In [ ]:
data

In [ ]:
random.shuffle(data)

In [ ]:
class Data():
    def __init__(self,matrix,type) -> None:
        self.x = matrix
        self.y =type
        

In [ ]:
data[0].path

In [ ]:
def getdata(data,batch_size):
    while True:
        x, y= [],[] 
        for i in range(len(data)):
            img=load_img(data[i].path,color_mode='rgb',target_size=(128,128),grayscale=False)
            matrix = img_to_array(img)
            x.append(matrix)
            if data[i].type=="chickenpox":
                y.append(0)
            if data[i].type=="cowpox":
                y.append(1)
            if data[i].type=="healthy":
                y.append(2)
            if data[i].type=="measles":
                y.append(3)
            if data[i].type=="monkeypox":
                y.append(4)
            if data[i].type=="smallpox":
                y.append(5)
            if len(y) == batch_size:                    
                        yield np.array(x),np.array(y)
                        x, y= [],[] 


In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*0.7)+2
val_size = int(len(data)*0.15)
test_size = int(len(data)*0.15)
print(f"train size: {train_size}, val size: {val_size}, test size: {test_size}, sum: {train_size+test_size+val_size}")

In [ ]:
train_size/64

In [ ]:
train_size%64

In [ ]:
def get_model():
    inp = keras.layers.Input((None, None ,3))
    base_model= tf.keras.applications.efficientnet.EfficientNetB5(include_top=False, weights="imagenet", pooling='max')(inp)
    base_model.trainable=True
   
    x=BatchNormalization()(base_model)
    x = keras.layers.Dense(256,activation='relu')(x)
    x=Dropout(rate=.4, seed=123)(x)       
    out = keras.layers.Dense(len(names), 'softmax')(x)
    model = keras.models.Model(inp, out)
    model.compile(loss="sparse_categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),metrics=["accuracy"])
    return model

In [ ]:
# j=0
# train =[]

# print(next(getdata(data,29646,64))[1])


In [ ]:
train = data[:train_size]
val=data[train_size:train_size+val_size]
test=data[train_size+val_size:]

In [ ]:
next(getdata(train,32))

In [ ]:
def Visualiztion(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()


In [ ]:

model =get_model()

history=model.fit(getdata(train,32),epochs=50
,callbacks = [keras.callbacks.EarlyStopping(monitor = 'loss', patience = 2, restore_best_weights = True)],verbose = 1, validation_steps = len(val)//32 ,                        
                                    steps_per_epoch = len(train)//32,validation_data = getdata(val, 32) )    
Visualiztion(history)

In [ ]:
model.evaluate(getdata(test,32))

In [ ]:
model.save("model1.model")